In [7]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time
import csv
import json
import re

## Authentication

In [8]:
CLIENT_ID = 'd8415d5149674a91a7e81de017c789d7'
CLIENT_SECRET = '576b0598e8014aeb93f7bc1ab69ac410'

client_credentials_manager = SpotifyClientCredentials(CLIENT_ID, CLIENT_SECRET)
spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

## Playlists

In [9]:
PARAMETERS_FILE = '../parameters.json'

with open(PARAMETERS_FILE) as f:
    parameters = json.load(f)

pattern = r'/playlist/([^/?]+)'

playlist_id = list()

for url in parameters['playlist_id']:
    match = re.search(pattern, url)

    if match:
        playlist_id.append(match.group(1))

playlist_id

['3tTo8cKpIQbLJ3MTzgEVjV', '1Kzd8jonR1022i8dL8eDda', '3bxNKleDijOT6Did05jINs']

### Mainstream Genres

## Get Artist Data

In [10]:
def get_artist_genres(artist_id:str) -> list:
    artist = spotify.artist(artist_id)

    return artist['genres']


def get_related_artists_names(artist_id:str) -> list:
    related_artists = spotify.artist_related_artists(artist_id)

    related_artists_names = [related_artist['name'] for related_artist in related_artists['artists']]

    return related_artists_names

def get_related_artists_genres(artist_id:str, track_genres:list) -> set:
    related_artists = spotify.artist_related_artists(artist_id)

    related_artists_genres = set()

    for related_artist in related_artists['artists']:
        genres = set(related_artist['genres'])

        for genre in genres:
            related_artists_genres.add(genre)

        print(related_artists_genres)

    return list(related_artists_genres - set(track_genres))

def get_generic_genres(track_genres:list, track_related_genres:list) -> list:
    generic_genres = list()

    for track_genre in track_genres:
        for genre in parameters['generic_genres']:
            if genre in track_genre:
                generic_genres.append(genre)

    for track_related_genre in track_related_genres:
        for genre in parameters['generic_genres']:
            if genre in track_related_genre:
                generic_genres.append(genre)

    return generic_genres

## Get Track Data

In [11]:
def get_tracks_id(playlist_id:str) -> list:  
    tracks_id = list()
    playlist_items = spotify.playlist_items(playlist_id)

    for item in playlist_items['items']:
        track = item['track']
        tracks_id.append(track['id'])

    return tracks_id

def get_track_data(track_id:str) -> dict:
    metadata = spotify.track(track_id)
    audio_features = spotify.audio_features(track_id)

    artist_id = metadata['album']['artists'][0]['id']
    track_genres = get_artist_genres(artist_id)
    related_track_genres = get_related_artists_genres(artist_id, track_genres)


    return {
        'id': metadata['id'],
        'name': metadata['name'],
        'album': metadata['album']['name'],
        'artist': metadata['album']['artists'][0]['name'],
        'genres': track_genres,
        'related_artists': get_related_artists_names(artist_id),
        'related_genres': related_track_genres,
        'generic_genres': get_generic_genres(track_genres, related_track_genres),
        'release_date': metadata['album']['release_date'],
        'duration_ms': metadata['duration_ms'],
        'popularity': metadata['popularity'],
        'acousticness': audio_features[0]['acousticness'],
        'danceability': audio_features[0]['danceability'],

        'energy': audio_features[0]['energy'],
        'instrumentalness': audio_features[0]['instrumentalness'],
        'key': audio_features[0]['key'],
        'liveness': audio_features[0]['liveness'],
        'loudness': audio_features[0]['loudness'],
        'mode': audio_features[0]['mode'],
        'speechiness': audio_features[0]['speechiness'],
        'tempo': audio_features[0]['tempo'],
        'time_signature': audio_features[0]['time_signature'],
        'valence': audio_features[0]['valence'],
    }

In [12]:
data = dict()

for _id in playlist_id:
    tracks_id = get_tracks_id(_id)
    tracks_data = list()

    for track_id in tracks_id:
        time.sleep(.5)
        tracks_data.append(get_track_data(track_id))
        
    data[_id] = tracks_data

{'supergroup', 'classic rock', 'rock', 'british blues', 'hard rock', 'blues rock', 'folk rock', 'album rock', 'psychedelic rock', 'singer-songwriter'}
{'supergroup', 'classic rock', 'rock', 'british blues', 'country rock', 'hard rock', 'blues rock', 'folk rock', 'album rock', 'british invasion', 'psychedelic rock', 'singer-songwriter'}
{'supergroup', 'classic rock', 'rock', 'british blues', 'country rock', 'hard rock', 'blues', 'electric blues', 'blues rock', 'folk rock', 'album rock', 'british invasion', 'psychedelic rock', 'singer-songwriter'}
{'supergroup', 'classic rock', 'rock', 'british blues', 'country rock', 'hard rock', 'blues', 'electric blues', 'blues rock', 'mellow gold', 'folk rock', 'album rock', 'british invasion', 'psychedelic rock', 'heartland rock', 'singer-songwriter'}
{'classic rock', 'rock', 'blues rock', 'album rock', 'psychedelic rock', 'heartland rock', 'electric blues', 'proto-metal', 'blues', 'soft rock', 'singer-songwriter', 'mellow gold', 'folk rock', 'briti

In [13]:
for key,value in data.items():
    print(key, value)
    with open(f"../data/metadata/{key}.json", "w") as f:
        json.dump(value, f, indent=4)

3tTo8cKpIQbLJ3MTzgEVjV [{'id': '4sQdW03IQh3tsZeBELdt5G', 'name': 'Foxey Lady', 'album': 'Are You Experienced', 'artist': 'Jimi Hendrix', 'genres': ['acid rock', 'album rock', 'classic rock', 'hard rock', 'proto-metal', 'psychedelic rock', 'rock'], 'related_artists': ['Cream', 'The Yardbirds', 'Canned Heat', 'Derek & The Dominos', 'Steppenwolf', 'Janis Joplin', 'Ten Years After', 'Free', 'Jefferson Airplane', 'The Doors', 'The Animals', 'Big Brother & The Holding Company', 'Stevie Ray Vaughan', 'Rory Gallagher', 'Jim Morrison', 'Traffic', 'John Mayall & The Bluesbreakers', 'Allman Brothers Band', 'Albert King', 'The Who'], 'related_genres': ['roots rock', 'southern rock', 'blues rock', 'heartland rock', 'texas blues', 'soul blues', 'electric blues', 'jam band', 'blues', 'soft rock', 'singer-songwriter', 'traditional blues', 'mellow gold', 'symphonic rock', 'folk rock', 'british invasion', 'progressive rock', 'british blues', 'supergroup', 'country rock', 'folk', 'protopunk', 'classic ca

In [14]:
data

{'3tTo8cKpIQbLJ3MTzgEVjV': [{'id': '4sQdW03IQh3tsZeBELdt5G',
   'name': 'Foxey Lady',
   'album': 'Are You Experienced',
   'artist': 'Jimi Hendrix',
   'genres': ['acid rock',
    'album rock',
    'classic rock',
    'hard rock',
    'proto-metal',
    'psychedelic rock',
    'rock'],
   'related_artists': ['Cream',
    'The Yardbirds',
    'Canned Heat',
    'Derek & The Dominos',
    'Steppenwolf',
    'Janis Joplin',
    'Ten Years After',
    'Free',
    'Jefferson Airplane',
    'The Doors',
    'The Animals',
    'Big Brother & The Holding Company',
    'Stevie Ray Vaughan',
    'Rory Gallagher',
    'Jim Morrison',
    'Traffic',
    'John Mayall & The Bluesbreakers',
    'Allman Brothers Band',
    'Albert King',
    'The Who'],
   'related_genres': ['roots rock',
    'southern rock',
    'blues rock',
    'heartland rock',
    'texas blues',
    'soul blues',
    'electric blues',
    'jam band',
    'blues',
    'soft rock',
    'singer-songwriter',
    'traditional blues',